# Parsing Cobol files using Snowpark

Here is a quick gist for parsing cobol file (dcm), which is present in an external stage (ex: S3).

## Solution overview
Snowpark can read files from the stage, [doc](https://docs.snowflake.com/en/LIMITEDACCESS/snowpark-python.html#working-with-files-in-a-stage).It is also easy to read cobol file using coboljsonifier, [doc](https://pypi.org/project/coboljsonifier/).
With this mind, I wanted to see what would it take Snowpark to read the cobol file, which is stored in an external stage and store the records into a Snowflake table.

The cobol file is read and its returned as JSON format. 

**Pre-requisite**
 - the external stage should be enabled as a directory table.
 
### Code Logic
 - Connect to Snowflake
 - Retreive the list of dicom files hosted in the external stage.
 - Import each cobol copybook/data file into session
 - Import python additional libraries into session
 - Create the UDF, which would parse the dicom file.
 - Iterate the stage directory table, but invoking the above defined udf.
 - If successful add the data into a staging table.

### Hurdles
Before rushing into the code, the steps was not simple as originally thought. The following are some hurdles that I had to overcome, to make the solution work.
#### Coboljsonifier library
The coboljsonifier library is not part of Snowflake python packages. Hence it needs to 
be imported dynamically. To overcome this 
 - we upload the library in the external stage
 - at runtime, we extract the libraries into a temporary folder
 - we add the folder to system path, this allows the python classes to be imported.

#### Staging cobol definition and data file
Currently python udf, cannot read files from the stage unless it is part of the import. To overcome this, I iterate through the stage folder where the dcm files are present and add each file as an import. 

## Limitations 
 - Please read the coboljsonifier [docs](https://github.com/jrperin/cobol-copybook.jsonifier), understand the process.
 - There could be some limitations on the number of files that can be imported and or total combined size of files.
 - This code is a temporary hack till python file stream access is in place.
 - The number of records in the data file can cause issue. particularly worried about the 16mb limit. In this a different strategy might need to done, which i can discuss.

## Thoughts
What I am demonstrating here is the possibility of parsing cobol files using Python. Hence you would need to
fine tune or re-implement the parsing logic, as you see fit. Also I am not demonstrating the process of building the
cobol definition file, for this please refer to coboljsonifier doc or the example [examples/README.md](https://raw.githubusercontent.com/jrperin/cobol-copybook.jsonifier/master/examples/README.md).

let me know how if this helps out, it will be great!!

---

In [1]:
# Initialize the Snowpark session
import os ,json
import dotenv 
import pandas as pd
from snowflake.snowpark import Session

#Load the login information from env file
dotenv.load_dotenv('./sflk.env')

#Create a snowpark session
connection_parameters = {
  "account": os.getenv('DEMO_ACCOUNT'),
  "user": os.getenv('DEMO_USER'),
  "password": os.getenv('DEMO_PWD'),
  "role": "sysadmin",
  "warehouse": os.getenv('DEMO_WH'),
  "database": os.getenv('DEMO_DB'),
  "schema": os.getenv('DEMO_SCH')
}

session = Session.builder.configs(connection_parameters).create()
#print(session.sql("select current_account() ,current_warehouse(), current_database(), current_schema()").collect())

In [4]:

# Iterate the stage 'stg_hl7data', specifically the folder 'datasets/cobol' and retrieve the list of files
stage_name = '@stg_hl7data'

session.sql(f'''alter stage stg_hl7data refresh;''').collect()

data_files = session.sql(f'''
    select 
        concat('{stage_name}/' ,relative_path) as full_path
    from directory({stage_name} )
    where relative_path like 'datasets/cobol/%';
''').collect()

# Clear any previous imports
session.clear_imports()

# Import each file into the session
for data_fl_row in data_files:
    fl_path = f'{data_fl_row[0]}'
    session.add_import(fl_path)
    
# Add the required libraries
libs_to_extract = ['coboljsonifier-1.0.3.tar.gz' ,'simplejson-3.17.6.tar.gz']
for lib in libs_to_extract:
    session.add_import(f'{stage_name}/pyfn_lib/{lib}')

# List out the imports, for debugging purposes.
session.get_imports()

['@stg_hl7data/datasets/cobol/ASCII_BOOK.cob',
 '@stg_hl7data/datasets/cobol/ASCII_DATA.bin',
 '@stg_hl7data/datasets/cobol/EBCDIC_BOOK.cob',
 '@stg_hl7data/datasets/cobol/EBCDIC_DATA.bin',
 '@stg_hl7data/pyfn_lib/coboljsonifier-1.0.3.tar.gz',
 '@stg_hl7data/pyfn_lib/simplejson-3.17.6.tar.gz']

In [21]:
##
# Define the udf, which would parse the cobol file
#

from snowflake.snowpark.udf import *
from snowflake.snowpark.types import Variant;
from snowflake.snowpark.functions import *
from snowflake.snowpark import *


'''
The UDF has the following paramter:
 - p_copybook_def_path : the full path to the copybook structure file. ex: @stg_hl7data/datasets/cobol/ASCII_BOOK.cob
 - p_coboldata_path : the full path to the cobol data file. ex: @stg_hl7data/datasets/cobol/ASCII_DATA.bin
'''
@udf(session=session ,name="cobolparser_snowpy" ,replace=True)
def cobolparser_snowpy(p_copybook_def_path: str ,p_coboldata_path: str) -> Variant:
    import os ,sys ,json ,tarfile
    import importlib.util
    from pathlib import Path

    # Extract the third party libraries into tmp folder and dynamically import
    IMPORT_DIR = sys._xoptions["snowflake_import_directory"]
    TARGET_FOLDER = f'/tmp/cobolparser_snowpy' + str(os.getpid())    
    libs_to_extract = ['coboljsonifier-1.0.3.tar.gz' ,'simplejson-3.17.6.tar.gz']
    for lib in libs_to_extract:
        PACKAGE_FNAME = lib.replace('.tar.gz', '')
        TARGET_LIB_PATH = f'{TARGET_FOLDER}/{PACKAGE_FNAME}/'
        Path(f'{TARGET_LIB_PATH}').mkdir(parents=True, exist_ok=True)
        
        #detect if the library is tar archived or zip archived
        #and extract accordingly
        if ('.tar.gz' in lib):
            tf = tarfile.open(f'{IMPORT_DIR}{lib}')
            tf.extractall(f'{TARGET_FOLDER}')
            
        elif ('.zip' in lib):
            with zipfile.ZipFile(f'{IMPORT_DIR}{lib}', 'r') as zip_ref:
                zip_ref.extractall(TARGET_FOLDER)
    
        #Add the extracted folder to sys path
        sys.path.insert(0 ,TARGET_LIB_PATH )
        
        #nasty coboljsonifier code organization.
        if ('coboljsonifier' in lib):
            sys.path.insert(0 ,f'{TARGET_LIB_PATH}/src' )
        
    #Import should be done, only after inserting the target_lib_path into the path
    import simplejson
    from coboljsonifier.copybookextractor import CopybookExtractor
    from coboljsonifier.parser import Parser
    from coboljsonifier.config.parser_type_enum import ParseType

    # an utility method to list the content of the 
    # a directory. meant for debugging needs. for ex: to find the
    # list of files in the import directory
    def list_directory(p_dir):
        fls = []
        for root, dirs, files in os.walk(p_dir):
            for file in files:
                fls.append(f'{root}{dirs}{file}')
        
        dat = {}
        dat['files'] = fls
        return json.dumps(dat)

    # The udf to handle the parsing
    def udf(p_copybook_def_path ,p_coboldata_path):
        #TODO wrap this in a try/catch for better code
        
        #Warning :
        #The code demoed here is based of 'examples/ascii_parser_test.py' from the
        #coboljsonifier project. You might need to modify the functionality based 
        #of what you are seeing in the field.
        copybook_def_full_path = f'{IMPORT_DIR}{p_copybook_def_path}'
        coboldata_full_path = f'{IMPORT_DIR}{p_coboldata_path}'
        
        # Build the Parser
        dict_structure = CopybookExtractor(copybook_def_full_path).dict_book_structure
        parser = Parser(dict_structure,  ParseType.FLAT_ASCII).build()
        
        #parse the records
        # Warnings : the no of records could seriously affect, you might need
        # to look into adopting a slightly different pattern of processing the
        # data, based of what you are seeing in the field.
        records = []
        i = 0
        ''' Important Note! 
            ebcdic file: Open the file with rb "read binary" and f2.read(size)
            ascii file : Open the file with  r "read text" and f2.readline()
        '''
        with open(coboldata_full_path, 'r') as f2:
            while True:
                row = {}
                # ASCII
                data = f2.readline()
                if not data:
                    # got eof
                    break
                
                i += 1
                row['Registry'] = i
                
                if data[0:2] == "02":         # for ASCII
                    parser.parse(data)
                    dict_value = parser.value
                    
                    ## ALERT: Don't use json.dumps. It doesn't threat Decimal formats - float for monetary values
                    # Use simplejson instead, it has support for Decimals
                    row['data'] = simplejson.dumps(dict_value)
                    
                    row['status'] = 'success'
                else:
                    row['status'] = f'failure : Registry type {data[0:2]} not processed'
                
                records.append(row)
                
        return records
    
    # -------------- MAIN ---------------
    return udf(p_copybook_def_path ,p_coboldata_path)
    # return list_directory(TARGET_FOLDER)

In [23]:
df = session.sql('''select 
    cobolparser_snowpy('ASCII_BOOK.cob' ,'ASCII_DATA.bin') as cobol_parsed
''')

df.show()

------------------------------------------------------
|"COBOL_PARSED"                                      |
------------------------------------------------------
|[                                                   |
|  {                                                 |
|    "Registry": 1,                                  |
|    "data": "{\"DATA1-REGISTRY-TYPE\": 2, \"DAT...  |
|    "status": "success"                             |
|  },                                                |
|  {                                                 |
|    "Registry": 2,                                  |
|    "data": "{\"DATA1-REGISTRY-TYPE\": 2, \"DAT...  |
|    "status": "success"                             |
|  },                                                |
|  {                                                 |
|    "Registry": 3,                                  |
|    "data": "{\"DATA1-REGISTRY-TYPE\": 2, \"DAT...  |
|    "status": "success"                             |
|  }      

In [21]:
#close the snowpark session
session.close()